In [1]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 2000
epochs = 100
batch_size = 32
optimizer = "adam"
#learning_rate = 0.001
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo

elenco_features = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX",
    "ATR",
    "DM_OSC",
    "ADX"
]
elenco_targets = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features)
n_targets = len(col_targets)

def crea_modello(n_timesteps, giorni_previsione, n_targets, n_features):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(70, input_shape=(n_timesteps, n_features)))
        model.add(RepeatVector(giorni_previsione))

        # lstm-2
        model.add(LSTM(90, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.3))
        # lstm-3
        model.add(LSTM(130, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.3))
        # lstm-4
        model.add(LSTM(70, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.4))

        # Aggiunta di layer Dense
        model.add(TimeDistributed(Dense(70, activation='relu')))
        model.add(Dropout(0))

        model.add(TimeDistributed(Dense(n_targets, activation='linear')))

        model.compile(optimizer="adam",
                      loss='mean_absolute_percentage_error')

        return model

from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import RobustScaler
from sklearn.base import clone
scaler = PowerTransformer()
#scaler = RobustScaler()



In [2]:
print("Importa librerie", end="", flush=True)

#from keras.losses import Huber
#print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from tensorflow.keras.layers import Dropout
print(".", end="", flush=True)
from tensorflow.keras.layers import RepeatVector
print(".", end="", flush=True)
from tensorflow.keras.layers import TimeDistributed
print(".", end="", flush=True)
from keras.regularizers import l2
print(".", end="", flush=True)
from tensorflow.keras import Sequential
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, Dense
print(".", end="", flush=True)
#from tensorflow.keras.optimizers import Adam
#print(".", end="", flush=True)
#from sklearn.model_selection import train_test_split
#print(".", end="", flush=True)
from tensorflow.keras.models import load_model
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

if os.path.exists('LSTM.keras'):
    print("Caricamento modello esistente")
    model = load_model("LSTM.keras")
else:
    print("Creazione modello")
    model = crea_modello(n_timesteps, giorni_previsione, n_targets, n_features)


Importa librerie.............=
Download lista ticker
Caricamento modello esistente


In [3]:
print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

idx_val, X_val, Y_val = fx.to_XY(ticker_val, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)
X_val = X_val.reshape((-1, n_timesteps * n_features))
Y_val = Y_val.reshape((-1, giorni_previsione * n_targets))
X_scaler_val = clone(scaler)
Y_scaler_val = clone(scaler)
X_val = X_scaler_val.fit_transform(X_val)
Y_val = Y_scaler_val.fit_transform(Y_val)
X_val = X_val.reshape((-1, n_timesteps, n_features))
Y_val = Y_val.reshape((-1, giorni_previsione, n_targets))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for i_ticker in range (n_simboli_addestramento + 1):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    try:
        ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            ticker.index = ticker.index.date
            print("Calcolo indicatori ticker")
            ticker = fx.crea_indicatori(ticker)
            ticker.dropna(axis=0, inplace=True)

            print("Definizione features e target")
            idx, X, Y = fx.to_XY(ticker, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

            print("Scaler")
            X = X.reshape((-1, n_timesteps * n_features))
            Y = Y.reshape((-1, giorni_previsione * n_targets))

            #X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y, idx, test_size=0.2, random_state=seed)
            X_train = X
            Y_train = Y

            X_scaler = clone(scaler)
            Y_scaler = clone(scaler)
            #X_scaler.fit(X_train)
            #Y_scaler.fit(Y_train)
            X_train = X_scaler.fit_transform(X_train)
            #X_test = X_scaler.fit_transform(X_test)
            Y_train = Y_scaler.fit_transform(Y_train)
            #Y_test = Y_scaler.fit_transform(Y_test)

            X_train = X_train.reshape((-1, n_timesteps, n_features))
            #X_test = X_test.reshape((-1, n_timesteps, n_features))
            Y_train = Y_train.reshape((-1, giorni_previsione, n_targets))
            #Y_test = Y_test.reshape((-1, giorni_previsione, n_targets))

            print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
            model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
            model.save("LSTM.keras")
    except Exception as e:
        print(e)
        continue


Download dati ticker validazione
Calcolo indicatori ticker validazione
1 di 2000: Ticker HMN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 23s 122ms/step - loss: 24.0306 - val_loss: 58.5437 - lr: 0.0010
Epoch 2/100
103/103 [==============================] - 9s 85ms/step - loss: 23.2628 - val_loss: 51.7361 - lr: 0.0010
Epoch 3/100
103/103 [==============================] - 8s 79ms/step - loss: 20.7059 - val_loss: 50.9203 - lr: 0.0010
Epoch 4/100
103/103 [==============================] - 8s 79ms/step - loss: 19.2594 - val_loss: 51.9377 - lr: 0.0010
Epoch 5/100
103/103 [==============================] - 8s 76ms/step - loss: 18.6031 - val_loss: 55.2367 - lr: 0.0010
Epoch 6/100
103/103 [==============================] - 8s 80ms/step - loss: 18.1953 - val_loss: 57.2424 - lr: 0.0010
Epoch 7/100
103/103 [==============================] - 8s 77ms/step - l

60/60 [==============================] - 5s 91ms/step - loss: 27.0436 - val_loss: 68.9866 - lr: 1.0000e-04
Epoch 7/100
60/60 [==============================] - 5s 91ms/step - loss: 27.3431 - val_loss: 66.8101 - lr: 1.0000e-04
Epoch 8/100
60/60 [==============================] - 6s 97ms/step - loss: 26.3337 - val_loss: 68.2473 - lr: 1.0000e-04
Epoch 9/100
60/60 [==============================] - 5s 90ms/step - loss: 25.9897 - val_loss: 66.4077 - lr: 1.0000e-04
Epoch 10/100
60/60 [==============================] - 6s 94ms/step - loss: 25.8153 - val_loss: 66.9486 - lr: 1.0000e-04
Epoch 11/100
60/60 [==============================] - 6s 98ms/step - loss: 25.8000 - val_loss: 68.7714 - lr: 1.0000e-04
6 di 2000: Ticker ESTA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
36/36 [==============================] - 4s 115ms/step - loss: 15.6278 - val_loss: 50.6172 - lr: 1.0000e-04
Epoch 2/100
36/36 [==

58/58 [==============================] - 5s 93ms/step - loss: 29.0651 - val_loss: 44.9655 - lr: 1.0000e-04
Epoch 3/100
58/58 [==============================] - 5s 89ms/step - loss: 26.7083 - val_loss: 45.0253 - lr: 1.0000e-04
Epoch 4/100
58/58 [==============================] - 5s 86ms/step - loss: 26.3519 - val_loss: 44.2373 - lr: 1.0000e-04
Epoch 5/100
58/58 [==============================] - 5s 84ms/step - loss: 25.2922 - val_loss: 44.2864 - lr: 1.0000e-04
Epoch 6/100
58/58 [==============================] - 6s 96ms/step - loss: 25.1703 - val_loss: 43.7650 - lr: 1.0000e-04
Epoch 7/100
58/58 [==============================] - 5s 91ms/step - loss: 24.3493 - val_loss: 44.3098 - lr: 1.0000e-04
Epoch 8/100
58/58 [==============================] - 5s 88ms/step - loss: 24.4221 - val_loss: 44.2482 - lr: 1.0000e-04
Epoch 9/100
58/58 [==============================] - 5s 85ms/step - loss: 23.5501 - val_loss: 45.6272 - lr: 1.0000e-04
Epoch 10/100
58/58 [==============================] - 5s 85m

Epoch 12/100
65/65 [==============================] - 5s 81ms/step - loss: 26.1942 - val_loss: 45.9852 - lr: 1.0000e-04
Epoch 13/100
65/65 [==============================] - 5s 80ms/step - loss: 27.7683 - val_loss: 46.1224 - lr: 1.0000e-04
15 di 2000: Ticker MPWR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 8s 75ms/step - loss: 14.5901 - val_loss: 44.2496 - lr: 1.0000e-04
Epoch 2/100
103/103 [==============================] - 8s 75ms/step - loss: 11.8004 - val_loss: 47.0913 - lr: 1.0000e-04
Epoch 3/100
103/103 [==============================] - 7s 70ms/step - loss: 10.7193 - val_loss: 49.3306 - lr: 1.0000e-04
Epoch 4/100
103/103 [==============================] - 8s 73ms/step - loss: 9.9892 - val_loss: 51.6276 - lr: 1.0000e-04
Epoch 5/100
103/103 [==============================] - 7s 72ms/step - loss: 9.4620 - val_loss: 56.6450 - lr: 1.0000e-04
E

Epoch 7/100
103/103 [==============================] - 7s 72ms/step - loss: 17.9307 - val_loss: 52.7354 - lr: 1.0000e-04
Epoch 8/100
103/103 [==============================] - 7s 73ms/step - loss: 17.4265 - val_loss: 53.1828 - lr: 1.0000e-04
Epoch 9/100
103/103 [==============================] - 7s 71ms/step - loss: 17.1077 - val_loss: 52.7532 - lr: 1.0000e-04
Epoch 10/100
103/103 [==============================] - 7s 72ms/step - loss: 17.3271 - val_loss: 53.3232 - lr: 1.0000e-04
Epoch 11/100
103/103 [==============================] - 9s 83ms/step - loss: 16.8017 - val_loss: 53.4833 - lr: 1.0000e-04
20 di 2000: Ticker CPK
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
103/103 [==============================] - 8s 75ms/step - loss: 15.8643 - val_loss: 48.3375 - lr: 1.0000e-04
Epoch 2/100
103/103 [==============================] - 8s 73ms/step - loss: 15.5606 - val_loss: 49.3402 - lr: 1.0000e

...


1 Failed download:
['CBTX']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
470 di 2000: Ticker BATRA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=32
Epoch 1/100
54/54 [==============================] - 6s 104ms/step - loss: 48.6302 - val_loss: 42.6323 - lr: 1.0000e-04
Epoch 2/100
54/54 [==============================] - 6s 106ms/step - loss: 37.1715 - val_loss: 42.4052 - lr: 1.0000e-04
Epoch 3/100
54/54 [==============================] - 5s 100ms/step - loss: 33.6225 - val_loss: 42.0568 - lr: 1.0000e-04
Epoch 4/100
54/54 [==============================] - 6s 105ms/step - loss: 33.0637 - val_loss: 42.8301 - lr: 1.0000e-04
Epoch 5/100
54/54 [==============================] - 5s 100ms/step - loss: 37.0756 - val_loss: 42.3575 - lr: 1.0000e-04
Epoch 6/100
54/54 [==============================] - 5s 98ms/step - loss: 31.0426 - val_loss: 42.6009 - lr: 1.0000e-04
Epoch 7/100
54/54 [==============================] - 5s 98ms/step - loss: 31.116


1 Failed download:
['1NOKIA']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
474 di 2000: Ticker EXPE
Download dati ticker
Calcolo indicatori ticker



KeyboardInterrupt



In [ ]:

print("Download dati ticker per previsione")
dati_previsione = yf.download(simbolo_test, start='2010-01-01', end='2023-12-31', progress=False)
dati_previsione.index = dati_previsione.index.date
dati_previsione = fx.crea_indicatori(dati_previsione)
dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

indice_prev, X_prev, Y_prev = fx.to_XY(dati_previsione, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=False)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione * n_targets)
X_scaler_prev = clone(scaler)
Y_scaler_prev = clone(scaler)
X_prev = X_scaler_prev.fit_transform(X_prev)
Y_prev = Y_scaler_prev.fit_transform(Y_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione, n_targets)

pred = model.predict(X_prev)
#pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
X_prev = X_scaler_prev.inverse_transform(X_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)

pred = pred.reshape(-1, giorni_previsione * n_targets)
pred = Y_scaler_prev.inverse_transform(pred)
pred = pred.reshape(-1, giorni_previsione, n_targets)

col_analisi = "EMA_50"

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import yfinance as yf

def grafico1(dati_previsione, pred):
    dati_previsione = dati_previsione.loc[indice_prev]
    new_dates = pd.bdate_range(start=dati_previsione.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)
    dati_previsione = pd.concat([dati_previsione, df_new])
    risultato = pd.DataFrame({"Previsione": pred[:, giorni_previsione-1, col_targets[col_analisi]].round(2)}, index=dati_previsione.index[giorni_previsione:])
    risultato = pd.concat([dati_previsione[col_analisi], risultato], axis=1)
    
    target = go.Scatter(
        x = risultato.index,
        y = risultato[col_analisi],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 250, .9)'),
        name = 'Previsione'
    )
    '''
    err_meno = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] - err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        connectgaps = False,
        name = 'err-'
    )                
    err_piu = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] + err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        fill='tonexty',
        fillcolor='rgba(0, 0, 250, .2)', # Puoi cambiare il valore di alpha per regolare la trasparenza
        connectgaps = False,
        name = 'err+'
    )
    '''
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    #fig.add_trace(err_meno, row=1, col=1)
    #fig.add_trace(err_piu, row=1, col=1)
    pyo.plot(fig, filename="grafico1.html", auto_open=True)




In [ ]:
def grafico2(dati_previsione, pred):
    dati2 = dati_previsione.dropna(axis=0).iloc[-60:]
    new_dates = pd.bdate_range(start=dati2.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)

    val_prev2 = (pred[-1, i, col_targets[col_analisi]] for i in range(giorni_previsione))
    prev2 = pd.DataFrame(val_prev2, columns=["Previsione"], index=df_new.index)
    riga_iniziale = pd.DataFrame({"Previsione": [dati2[col_analisi].iloc[-1]]}, index=[dati2.index[-1]])
    prev2 = pd.concat([riga_iniziale, prev2], axis=0)
    
    target = go.Scatter(
        x = dati2.index,
        y = dati2[col_analisi].round(2),
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = prev2.index,
        y = prev2['Previsione'].round(2),
        mode = 'lines',
        line = dict(color='rgba(0, 0, 250, .9)'),
        name = 'Previsione'
    )
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    pyo.plot(fig, filename="grafico2.html", auto_open=True)



In [ ]:
grafico1(dati_previsione, pred)

In [ ]:
grafico2(dati_previsione, pred)